In [5]:
import pandas as pd
from prophet import Prophet
from neuralprophet import NeuralProphet# as Prophet
from neuralprophet import set_log_level
set_log_level("ERROR")

from dotenv import load_dotenv
from pathlib import Path
import os

# from ezekial.prophet import Prophet
import yfinance as yf
import requests
# import base64
import pybase64 as base64
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from multiprocessing import Pool, cpu_count, Process#, freeze_support
import time
import warnings
import sys

In [6]:
# Arguements
warnings.filterwarnings("ignore")
load_dotenv()
url = os.getenv("DJANGO_SERVER_URL")
av_key = os.getenv("APLHA_VANTAGE_KEY")
av_key1 = os.getenv("APLHA_VANTAGE_KEY1")
av_key2 = os.getenv("APLHA_VANTAGE_KEY2")
av_key3 = os.getenv("APLHA_VANTAGE_KEY3")
av_key4 = os.getenv("APLHA_VANTAGE_KEY4")
av_key5 = os.getenv("APLHA_VANTAGE_KEY5")
av_key6 = os.getenv("APLHA_VANTAGE_KEY6")
ma_key = os.getenv("MARKETAUX_TOKEN")
ma_key1 = os.getenv("MARKETAUX_TOKEN2")
ma_key2 = os.getenv("MARKETAUX_TOKEN3")
ma_key3 = os.getenv("MARKETAUX_TOKEN4")
ma_key4 = os.getenv("MARKETAUX_TOKEN5")

weeeks_prior_data_end_date = int(sys.argv[1]) #260
forecast_days_ahead = int(sys.argv[2]) #90
ticker_path = sys.argv[3]#'data/S&P500 tickers.csv'
forecast_img_path = sys.argv[4]#'images/forecast_temp/forecast.png'
include_sentiment = sys.argv[5]
model_to_use = sys.argv[6]

In [7]:
# # Arguements
# warnings.filterwarnings("ignore")
# load_dotenv()
# url = os.getenv("DJANGO_SERVER_URL")
# url = os.getenv("DJANGO_SERVER_URL")
# av_key = os.getenv("APLHA_VANTAGE_KEY")
# av_key1 = os.getenv("APLHA_VANTAGE_KEY1")
# av_key2 = os.getenv("APLHA_VANTAGE_KEY2")
# av_key3 = os.getenv("APLHA_VANTAGE_KEY3")
# av_key4 = os.getenv("APLHA_VANTAGE_KEY4")
# av_key5 = os.getenv("APLHA_VANTAGE_KEY5")
# av_key6 = os.getenv("APLHA_VANTAGE_KEY6")
# ma_key = os.getenv("MARKETAUX_TOKEN")
# ma_key1 = os.getenv("MARKETAUX_TOKEN2")
# ma_key2 = os.getenv("MARKETAUX_TOKEN3")
# ma_key3 = os.getenv("MARKETAUX_TOKEN4")
# ma_key4 = os.getenv("MARKETAUX_TOKEN5")

# weeeks_prior_data_end_date = 12
# forecast_days_ahead = 90
# ticker_path = 'data/S&P500 tickers.csv'
# forecast_img_path = 'images/forecast_temp/forecast.png'
# include_sentiment = 'F'
# model_to_use = 'NP'

In [8]:
# print(weeeks_prior_data_end_date)
# print(forecast_days_ahead)
# print(ticker_path)
# print(forecast_img_path)
# print(include_sentiment)
# print(model_to_use)

In [14]:
def load_tickers(ticker_path):
    """
    Takes five hundred tickers, which is max calls of yfinance for public ip.
    """
    ticker_file_path = Path(ticker_path)
    ticker_df = pd.read_csv(ticker_file_path)
    prophet_ticker_list = ticker_df['Symbol'].head(500).to_list()
    return prophet_ticker_list

def django_get(url):
    response = requests.get(url)
    response_json = response.json()
    return response_json

def clear_django_sql_db(url):
    """
    Clears the sql db in the django api.
    "DJANGO_SERVER_URL" is provided in .env file.
    """

    try:
        django_api_items = django_get(url)

        if len(django_api_items) != 0:
            for x, index_id in enumerate(django_api_items):
                x += 1
                str_indx = str(index_id['id'])

                if x == 1:
                    url += str(str_indx)
                    requests.delete(url)
                    # print(f'entered if block\t{url}')
                else:
                    try:
                        url = url[:-len(str_indx)] + str_indx
                        requests.delete(url)
                        # print(f'entered else block\t{url}')
                    # For when the number of digits change
                    except:
                        url = url[:-len(str_indx) + 1] + str_indx
                        requests.delete(url)                    
                        # print(f'entered else block\t{url}')
            print(f"Django SQL DB cleared:\n\n{django_get(url)}")
        else:
            print(f"Django SQL DB Already empty:\n\n{django_get(url)}")
    except:
        print(f"Django server did not clear:\n\n{django_get(url)}")

def run_prophet(timeserie, forecast_ahead = forecast_days_ahead):
    model = Prophet(interval_width=0.95, changepoint_range=0.75, changepoint_prior_scale=0.05)
    # model = Prophet(yearly_seasonality=False,daily_seasonality=False)
    model.fit(timeserie)
    forecast = model.make_future_dataframe(periods=forecast_ahead, include_history=True)
    forecast = model.predict(forecast)
    return forecast

def run_nueral_prophet(timeserie, forecast_ahead = forecast_days_ahead):
    # m1 = NeuralProphet()
    # m1 = NeuralProphet(epochs=100)
    m1 = NeuralProphet(changepoints_range=0.75)
    # metrics = m1.fit(timeserie, freq="D", minimal=True)
    metrics = m1.fit(timeserie, freq="D")
    future = m1.make_future_dataframe(df=timeserie, periods=forecast_ahead, n_historic_predictions=True)
    forecast = m1.predict(future)
    forecast = forecast.rename(columns={'yhat1':'yhat'})
    return forecast

def make_plots(prediction, seri, adj_cl, idx):

    forecast_plot = pd.DataFrame()
    if model_to_use == 'FBP': forecast_plot = pd.merge(left=prediction, right=seri, on='ds', how='outer')
    else: forecast_plot = prediction
    ticker_sym = adj_cl
    
    if idx < 100:
        ma_key_valu = ma_key
    elif idx < 200:
        ma_key_valu = ma_key1
    elif idx < 300:
        ma_key_valu = ma_key2
    elif idx < 400:
        ma_key_valu = ma_key3
    else:
        ma_key_valu = ma_key4
        
    # SENTIMENT
    if include_sentiment == 'T':
        try:
            ma_url = f'https://api.marketaux.com/v1/news/all?symbols={ticker_sym}&filter_entities=true&group_similar=false&language=en&api_token={ma_key_valu}'
            setntiment_data = requests.get(ma_url).json()

            mean_score = 0
            for i in range(len(setntiment_data['data'])):
                mean_score += setntiment_data['data'][i]['entities'][0]['sentiment_score']

            mean_score /= len(setntiment_data['data'])
        except:
            mean_score = 'NULL'
    else:
        mean_score = 'NULL'
    
    ########################################################################
    ## ENCODE QUANTILES FOR FORECASTS USING PCT_CHANGE OF HISTORICAL DATA ##
    ########################################################################
    forecast_plot.index = forecast_plot['ds']
    yhat_descriptive_stats = forecast_plot[['y']].dropna().pct_change().dropna().dot([1000]).astype(int).describe()

    twentyfive_percent = float(yhat_descriptive_stats.iloc[4])
    fifty_percent = float(yhat_descriptive_stats.iloc[5])
    seventy_five_percent = float(yhat_descriptive_stats.iloc[6])

    def encode_quantiles (z):     
        if z < twentyfive_percent: z = 0
        elif z < fifty_percent: z = 1
        elif z < seventy_five_percent: z = 3
        else: z = 4
        return z

    weather_df = forecast_plot[['yhat']].pct_change().dropna().dot([1000]).astype(int)[-forecast_days_ahead:-(forecast_days_ahead - 10)].apply(encode_quantiles)
    weather_df = [[str(x)[-22:] for x in weather_df.index], [y for y in weather_df]]

    
    # MAKE FORECAST PLOTS
    forecast_plot = forecast_plot[-(forecast_days_ahead - 10) - 30:-(forecast_days_ahead - 10)]
    # forecast_plot.index = forecast_plot['ds']

    with plt.style.context('dark_background'):
        fig, ax = plt.subplots()
        forecast_plot[['y','yhat']].plot(ax=ax, figsize = (4,5), style=['o-','--'],color=['r','lightblue'],linewidth=1)
        ax.set_xlabel('Date', fontdict={'size':12})
        ax.set_ylabel('Price', fontdict={'size':12})
        ax.yaxis.set_major_formatter('${x:,.0f}')
        ax.legend(['Actual','Forecasted'], title= f'{ticker_sym}', loc='upper center', frameon=False)
        # ax.set_facecolor('xkcd:black')
    # plt.show()
    try:
        # new_forecast_img_path = forecast_img_path + F'_{ticker_sym}.png'
        # fig.savefig(new_forecast_img_path)
        fig.savefig(forecast_img_path)
        with open(forecast_img_path, "rb") as image2string:
        # with open(new_forecast_img_path, "rb") as image2string:
            converted_string = base64.b64encode(image2string.read())
        image2string.close()

        # JSON TO BE POSTED TO API
        json_post = {
            "ticker": f"{ticker_sym}",
            "time": f'{today}'[:16],
            "encoded_string": f"{converted_string}",
            "emoticons": f"{weather_df}",
            "sentiment": f"{mean_score}"
        }

        try:
            requests.post(url,json_post)
            print(f"+++ {ticker_sym} success!")
        except:
            print(f"\t--- {ticker_sym} failed ---")
    except:
        print(f"\t---# {ticker_sym} failed #---")
        print("\n\n\tyfinance API daily limit reached.")

In [10]:
start_time = time.time()
today = datetime.today()
start_date = today - timedelta(weeks=weeeks_prior_data_end_date)
start_date = start_date.replace(second=0, microsecond=0)

data = yf.download(load_tickers(ticker_path), start=start_date)
adj_close_df = data['Adj Close']
adj_close_df = adj_close_df.dropna(axis=1, how='all')
series = [adj_close_df[f'{col}'].reset_index().rename({'Date':'ds', f'{col}':'y'}, axis=1) for col in adj_close_df]


start_time00 = time.time()
p1 = Pool(cpu_count())

if model_to_use == 'FBP':
    predictions1 = list(tqdm(p1.imap(run_prophet, series), total=len(series)))

elif model_to_use == 'NP':
    predictions2 = list(tqdm(p1.imap(run_nueral_prophet, series), total=len(series)))

p1.close()
p1.join()
print("--- %s seconds ---" % (time.time() - start_time00))

[*********************100%***********************]  500 of 500 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted


  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

...

  0%|          | 0/104 [00:00<?, ?it/s]

Epoch[94/778]:  11%|█▏        | 89/778 [00:01<00:12, 57.06it/s]s]

  0%|          | 0/104 [00:00<?, ?it/s]

Epoch[187/778]:  24%|██▎       | 184/778 [00:03<00:10, 56.18it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



--- 972.8759214878082 seconds ---


In [15]:
# Attempt to clear the SQL DB if it fails initially before uploading the data again, 
# to reduce DB size for POST request on the APP side
try: clear_django_sql_db(url = url)
except: 
    try: clear_django_sql_db(url = url)
    except: error_msg = '\t##### Could not clear the Django SQL DB  #####'

if model_to_use == 'FBP':
    mapping_star = [(predictions1[i], series[i], adj_close_df.columns[i], i) for i in range(len(predictions1))]

elif model_to_use == 'NP':
    mapping_star = [(predictions2[i], series[i], adj_close_df.columns[i], i) for i in range(len(predictions2))]

start_time1 = time.time()

p = Pool(cpu_count())
p.starmap(make_plots, mapping_star)
p.close()
p.join()
print("--- %s seconds ---" % (time.time() - start_time1))

Django SQL DB Already empty:

[]
+++ AVGO success!
+++ CRL success!
+++ A success!
+++ CRM success!
+++ AAL success!
+++ CL success!
+++ AAP success!
+++ CSCO success!
+++ CBRE success!
+++ AAPL success!
+++ CCI success!
+++ CSX success!
+++ AVY success!
+++ AMP success!
+++ AWK success!
+++ CLX success!
+++ AXP success!
+++ AMT success!
+++ BKR success!
+++ ABBV success!
+++ AZO success!
+++ CMA success!
+++ BA success!
+++ CMCSA success!
+++ ABC success!
+++ AMZN success!
+++ CCL success!
+++ ABMD success!
+++ CDAY success!
+++ ABT success!
+++ CDNS success!
+++ ACN success!
+++ CME success!
+++ ANET success!
+++ CMG success!
+++ ADBE success!
+++ CDW success!
+++ ANSS success!
+++ CE success!
+++ ANTM success!
+++ CEG success!
+++ AON success!
+++ BAC success!
+++ AOS success!
+++ ADI success!
+++ BAX success!
+++ ADM success!
+++ BBWI success!
+++ ADP success!
+++ BBY success!
+++ ADSK success!
+++ BDX success!
+++ CMI success!
+++ BEN success!
+++ AES success!
+++ BIIB success!
++

In [16]:
try: print(error_msg)
except: print("\t+++++ The SQL DB was successfully cleared +++++")

print("--- %s seconds ---" % (time.time() - start_time))

	+++++ The SQL DB was successfully cleared +++++
--- 1730.3706755638123 seconds ---
